После того, как кластер spark создался, можно начать пользоваться более удобным интерфейсом и работать в Jupyter, который хостится уже на кластере.

Он немножко сломан по умолчанию, но на официальном форуме рассказали, что это легко починить:

Необходимо подключиться к головной машине через ssh, открыть файл `/usr/bin/anaconda/lib/python2.7/site-packages/nbformat/_version.py` и заменить 5 на 4.

После этого остается перезагрузить Jupyter через ambari.

Сессия спарка доступна в ноутбуке через переменную `spark`.
Для того, чтобы спарк "прогрелся" и начал выполнять запросы, создадим контекст, который нам впоследствии потребуется.

In [ ]:
sp = spark.sparkContext

In [ ]:
data = sp.textFile("wasb:///seclog/day_01.csv")

Количество строк в файле

In [ ]:
data.count()

Предыдущая задача решенная на спарке:

In [ ]:
def get_user_and_size(line):
    columns = line.split(',')
    user, size = columns[0], columns[8]

    return user, float(size)

result = data.map(get_user_and_size).reduceByKey(lambda x, y: x+y).values().mean()
print(result)

Можно заметить, что получилось гораздо приятнее и быстрее, чем класический MR.

Полный список операций, которые можно делать на спарке - здесь 
* https://spark.apache.org/docs/latest/rdd-programming-guide.html
* http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD

Считаем общее количество пользователей

In [ ]:
total_users = data.map(lambda x: x.split(',')[0]).distinct().count()
print(total_users)

Считаем общее количество пользователей, которые сидят ночью

In [ ]:
total_night_users = data.filter(lambda x: int(x.split(',')[2].split(':')[0]) < 6).map(lambda x: x.split(',')[0]).distinct()\
                    .count()
print(total_night_users)

Считаем 10 самых больших документа, которые скачивали пользователи

In [ ]:
def get_name_and_size(line):
    columns = line.split(',')
    name, size = columns[6], columns[8]
    return float(size), name

top_10_large_result = data.map(get_name_and_size).sortByKey(ascending=False).values()\
                      .zipWithIndex().filter(lambda x: x[1] < 10).keys().collect()
print(result)

Помимо того, что результат можно получить в Jupyter, его можно положить в HDFS

In [56]:
! hdfs dfs -mkdir -p /seclogres

In [ ]:
result = data.map(get_name_and_size).sortByKey(ascending=False).values()\
        .zipWithIndex().filter(lambda x: x[1] < 10).keys()

result.saveAsTextFile("wasb:///seclogres/top_10_requests.txt")

In [64]:
! hdfs dfs -cat /seclogres/top_10_requests.txt/part-00000

.txt
.txt
.txt
.txt
.txt
.txt
.txt
.txt
.txt
.txt


**Задача**

* Посчитать топ 10 самых посещаемых страниц (считаются только успешные запросы - код 200)
* Посчитать суммарное количество людей по часам (~гистограма)
* Посчитать среднее количество людей по часам (~гистограма)

In [ ]:
# DO IT

Вычисления можно также проводить и в более "ручном" режиме (примерно как в MR)

Ниже - вычисление среднего объема, который выкачивает каждый пользователь (смотри задачу выше), решенная немного другим подходом

In [ ]:
def mapper(line):
    columns = line.split(',')
    user, size = columns[0], columns[8]
    return user, float(size)

def sum_reducer(item):
    key, values = item
    result = 0
    for value in values:
        result += value
    return result, 1

def mean_reducer(item):
    result_key, values = item
    summ, count = 0, 0
    for current_summ, current_count in values:
        summ += current_summ
        count += current_count
    return summ / count


result = data.map(mapper).groupByKey().map(sum_reducer).groupBy(lambda x: 1).map(mean_reducer).collect()
print(result)

**Задача**

* Скачать все логи за год в кластер. Найти всех пользователей, которые заходили каждый день месяца с указанием - что за месяц (если такие есть). Если таких нет - найти пользователей, которые заходили наибольшее число раз в наибольшее количество дней (с указанием в какие месяца)

In [ ]:
# DO IT